In [1]:
import os
import re
os.chdir('../')

In [25]:
import json
import time
import requests
import numpy as np
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
  

In [4]:
from src.general_utils import util

In [5]:
cities = [
    'bengaluru',
    # 'national-capital-region-ncr',
]

In [26]:
class GetMovieDetails:
    def __init__(self, path_movie_url, movie_name, date, location):
        
        self.path_movie_url = path_movie_url   
        self.movie_name = movie_name 
        self.date = date
        self.location = location
        self.formatted_time = self._get_current_time()

    def _get_current_time(self):
        # Get the current time in seconds since the epoch
        current_time_seconds = time.time()

        # Convert seconds since the epoch to a time structure
        time_struct = time.localtime(current_time_seconds)

        # Format the time in 24-hour format (HH:MM:SS)
        formatted_time = time.strftime("%H:%M", time_struct)
        return formatted_time
    
    def _get_header(self):
        headers = {
                'Referer': 'https://in.bookmyshow.com/explore/movies-bengaluru?languages=hindi',
                'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
                'sec-ch-ua-mobile': '?0',
                'sec-ch-ua-platform': "Linux",
                'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
                }
        return headers
        
    
    def _get_chrome_driver(self):
        # driver = webdriver.Chrome(ChromeDriverManager().install())
        driver = webdriver.Chrome()
        driver.get(self.path_movie_url)
        return driver

    
    def convert_to_json(self, data_categories:str):
        return json.loads(data_categories)

    def get_price(self, data_json):
        return data_json.get('price')

    def get_desc(self, data_json):
        return data_json.get('desc')

    def get_availability_class(self, data_json):
        return data_json.get('availabilityClass')

    def get_availStatus(self, data_json):
        return data_json.get('availStatus')

    def get_processed_df(self):
        driver = self._get_chrome_driver()
        
        print(driver)
        theatre_element_list = driver.find_elements(By.CLASS_NAME, 'list')
        print(f'Number of theatres : {len(theatre_element_list)}')

        data = []

        for i, a in tqdm(enumerate(theatre_element_list)):
            try:
                titles = a.find_element(By.CLASS_NAME, '__title')
                listing_info = a.find_elements(By.CLASS_NAME, 'listing-info')
                showtimes = a.find_elements(By.CLASS_NAME, 'showtime-pill')

                for s in showtimes:

                    data_categories = self.convert_to_json(s.get_attribute('data-categories'))
                    timings = s.get_attribute('data-date-time')

                    temp_data = {
                        'cinema_title' : titles.text,
                        'timings': timings,
                        'latitude': a.get_attribute('data-lat'),
                        'longtitude' : a.get_attribute('data-lng'),

                        'price': self.get_price(data_categories),
                        'description': self.get_desc(data_categories),

                        'availability_class': self.get_availability_class(data_categories),
                        'avail_status': self.get_availStatus(data_categories),
                        'movie_name' : self.movie_name,
                        'date' : self.date,
                        'location' : self.location,
                        'checked_time' : self.formatted_time,
                    }
                    
                    data.append(temp_data)

                # b.find_element('showtime-pill')
            except:
                print(i)

        
        data_df = pd.DataFrame(data)

        return data_df
        

In [9]:
class GetMoviesList:
    def __init__(self, path_movie_list_url, location, date):
        self.date = date
        self.location = location
        self.path_movie_list_url = path_movie_list_url    
        self.movie_ahref_string = f'https://in.bookmyshow.com/{location}/movies'

    def _get_header(self):
        headers = {
                'Referer': 'https://in.bookmyshow.com/explore/movies-bengaluru?languages=hindi',
                'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
                'sec-ch-ua-mobile': '?0',
                'sec-ch-ua-platform': "Linux",
                'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
                }
        return headers

    def _get_chrome_driver(self):
        # driver = webdriver.Chrome(ChromeDriverManager().install())
        driver = webdriver.Chrome()
        driver.get(self.path_movie_list_url)
        
        # Scroll to buttom slowly to load all movies
        driver.execute_async_script(
            """
        count = 400;
        let callback = arguments[arguments.length - 1];
        t = setTimeout(function scrolldown(){
            console.log(count, t);
            window.scrollTo(0, count);
            if(count < (document.body.scrollHeight || document.documentElement.scrollHeight)){
              count+= 400;
              t = setTimeout(scrolldown, 1000);
            }else{
              callback((document.body.scrollHeight || document.documentElement.scrollHeight));
            }
        }, 1000);"""
        )
        return driver
    
    def parse_drive_to_soup(self, driver):
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        return soup
    
    def get_all_current_movie_list(self, soup):
        
        movie_soup_element_dict = {}
        
        movie_tags_list = [a for a in soup.find_all('a', href=True) if self.movie_ahref_string in a['href']]
        
        # Extract movie names from the URLs
        movie_names = []
        for link in movie_tags_list:
            # Extract the movie name from the URL
            movie_name_match = re.search(r'.*/movies/(.*)/ET.*', link['href'])
            if movie_name_match:
                movie_name = movie_name_match.group(1)
                movie_names.append(movie_name)

                movie_url = link['href']
                movie_url_basename = movie_url.split('/')[-1]
                
                movie_booking_url = f'https://in.bookmyshow.com/buytickets/{movie_name}-{self.location}/movie-bang-{movie_url_basename}-MT/{self.date}'

                movie_soup_element_dict[movie_name] = {
                    # 'element': link,
                    'url': movie_url,
                    'name': movie_name,
                    'booking_url' : movie_booking_url,
                    'date': self.date,
                    'location' : self.location,
                }
            
        # # Print the movie names
        # for movie_name in movie_names:
        #     print("Movie Name:", movie_name)

        
        print(f'Number of movies running: {len(movie_names)}')
        return movie_soup_element_dict
        
    def generate(self):
        driver = self._get_chrome_driver()
        soup = self.parse_drive_to_soup(driver)
        movie_soup_element_dict  = self.get_all_current_movie_list(soup)
        
        return movie_soup_element_dict

In [12]:
location = 'bengaluru'
date = '20230916'
path_movie_list_url = f'https://in.bookmyshow.com/explore/movies-{location}'
movie_list_instance = GetMoviesList(path_movie_list_url=path_movie_list_url, location=location, date=date)
movie_soup_element_dict = movie_list_instance.generate()

Number of movies running: 27


In [13]:
path_output = f'data/{location}_scrapped_{date}.json'
util.save_json(movie_soup_element_dict, path_output=path_output)

In [15]:
data_movie_list = []

for movie_name in tqdm(movie_soup_element_dict):
    path_movie_url = movie_soup_element_dict[movie_name]['booking_url']

    movie_details = GetMovieDetails(path_movie_url=path_movie_url,
                                    location=location, date=date,
                                    movie_name=movie_name,
                                    
                                    )
    movie_detail_df = movie_details.get_processed_df()
    data_movie_list.append(movie_detail_df)


  0%|          | 0/27 [00:00<?, ?it/s]

<selenium.webdriver.chrome.webdriver.WebDriver (session="4d51a17dd2c99c98f64192cf1df8a36c")>
Number of theatres : 48


0
1
2
3
4
5


48it [00:02, 19.49it/s]
  4%|▎         | 1/27 [00:18<07:51, 18.14s/it]

46
47
<selenium.webdriver.chrome.webdriver.WebDriver (session="86cb2cd6a6fee1c2ce91549be26649ab")>
Number of theatres : 97


...

0
1
2
3
4
5


12it [00:00, 46.45it/s]
 96%|█████████▋| 26/27 [07:59<00:15, 15.96s/it]

10
11
<selenium.webdriver.chrome.webdriver.WebDriver (session="416b5da3439ac3b4a1c5b2a9cb88a944")>
Number of theatres : 9


9it [00:00, 88.27it/s]
100%|██████████| 27/27 [08:16<00:00, 18.37s/it]

0
1
2
3
4
5
7
8


In [17]:
data_movie_list_pd = pd.concat(data_movie_list)

In [18]:
data_movie_list_pd

,title,timings,latitude,longtitude,price,description,availability_class,avail_status
0,Akash Cinemas: Laggere\nINFO,10:30 AM,13.0145,77.516,100.00,Silver Class,_available,3
1,Akash Cinemas: Laggere\nINFO,01:30 PM,13.0145,77.516,100.00,Silver Class,_available,3
2,Cinephile HSR Layout: PNR Felicity Mall Haralu...,10:15 AM,12.8963,77.6579,236.00,GOLD,_available,3
3,Cinephile HSR Layout: PNR Felicity Mall Haralu...,05:15 PM,12.8963,77.6579,236.00,GOLD,_available,3
4,Cinepolis: Binnypet Mall\nINFO,11:05 AM,12.9676,77.5584,150.00,PREMIUM,_available,3
...,...,...,...,...,...,...,...,...
0,"PVR: Nexus (Formerly Forum), Koramangala\nINFO",10:05 PM,12.9346,77.6111,510.00,RECLINER,_available,3
1,"PVR: Orion Mall, Dr Rajkumar Road\nINFO",02:00 PM,13.011,77.5551,410.00,RECLINER,_available,3
2,"PVR: Phoenix Marketcity Mall, Whitefield Road\...",09:30 PM,12.9973,77.6957,480.00,RECLINER,_available,3
3,"PVR: Vega City, Bannerghatta Road\nINFO",04:25 PM,12.9077,77.6012,410.00,RECLINER,_available,3


In [204]:
# # for holding the resultant list
# element_list = []
  
# for page in range(1, 3, 1):
    
#     page_url = "https://webscraper.io/test-sites/e-commerce/static/computers/laptops?page=" + str(page)
#     driver = webdriver.Chrome(ChromeDriverManager().install())
#     driver.get(page_url)
#     title = driver.find_elements_by_class_name("title")
#     price = driver.find_elements_by_class_name("price")
#     description = driver.find_elements_by_class_name("description")
#     rating = driver.find_elements_by_class_name("ratings")
  
#     for i in range(len(title)):
#         element_list.append([title[i].text, price[i].text, description[i].text, rating[i].text])
  
# print(element_list)
  
# #closing the driver
# driver.close()